# Exploring Russian Lexicon-based Tokenizer

- Top F1=0.94 with delimiting symbols added to the lexicon (less than Freedom-based top F1=1.0)
- Top F1 based on legth-driven parsing, not weight-driven or "hybrid" (legth-times-logweight-driven)
- Errors (0.6) are due to unknown words
- Not adding delimiters to lexicon drops top F1 to 0.79
- Adding threshold on word freqency does not improve F1
- Lexicon-based tokenization on spaceless text has F1=0.72 (comparable fo Chinese F1=0.83) with results explainable by lack of word stress articulation and speech pauses (expectedly can be improved based on alternative tokenization-trees maximizing the weight across entire tree)
- Precision of word discovery of Freedom-peak-based tokenizer is 1.0 (after correction all out-of-refernce-lexicon words), same as delimiter-based (1.0)


In [1]:
import os, sys
cwd = os.getcwd()
project_path = cwd[:cwd.find('pygents')+7]
if project_path not in sys.path: sys.path.append(project_path)
os.chdir(project_path) 

#from importlib import reload  # Python 3.4+

import pickle
import pandas as pd

#force reimport
if 'pygents.util' in sys.modules:
    del sys.modules['pygents.util']
if 'pygents.text' in sys.modules:
    del sys.modules['pygents.text']
if 'pygents.plot' in sys.modules:
    del sys.modules['pygents.plot']
if 'pygents.token' in sys.modules:
    del sys.modules['pygents.token']
if 'pygents.token_plot' in sys.modules:
    del sys.modules['pygents.token_plot']


from pygents.token import *
from pygents.text import *
from pygents.util import *
from pygents.plot import plot_bars, plot_dict, matrix_plot
from pygents.token_plot import *

In [2]:
path = 'data/corpora/Russian/'
test_df = pd.read_csv(os.path.join(path,'magicdata/zh_en_ru_100/CORPUS_ZH_EN_RU.txt'),delimiter='\t')
test_texts = list(test_df['ru'])
print(len(test_texts))
test_df[['ru']]

100


,ru
0,Как насчет медицинской страховки? В случае ваш...
1,"Для тех, у кого есть страховка, по договору ст..."
2,Необходимо осознать важность наличия страховки.
3,"На самом деле, это явление действительно очень..."
4,Этому поколению родителей действительно необхо...
...,...
95,Ресо предлагает не только страхование автомоби...
96,"Однако при покупке дома, за исключением первон..."
97,Этот вид финансовых вложений имеет определенны...
98,"Если ваша инвестиционная ориентация верна, то ..."


In [3]:
for text in test_texts:
    print(text)

Как насчет медицинской страховки? В случае вашей семьи, её можно оформить и взрослому и ребенку.
Для тех, у кого есть страховка, по договору страхования они получат компенсацию в размере 300 тысяч рублей.
Необходимо осознать важность наличия страховки.
На самом деле, это явление действительно очень распространено, например, для страхования от несчастных случаев, чем больше вы покупаете, тем больше страхуете.
Этому поколению родителей действительно необходимо покупать страховку.
Ну, прямо сейчас действительно желательно купить страховку.
Машину нужно покупать в полном объеме, а дом можно купить в кредит.
Вы можете купить страховку, страховка, конечно же, делится на множество категорий.
Медицинская страховка очень важна.
Эту часть денег платит страховая компания.
Послушайте, я не знаю, слышали ли вы когда-нибудь об этом, это страховка в Сбере.
Покупка дома на самом деле является инвестицией.
Вы когда-нибудь узнавали об обучении в Альфе?
Если он депонирован в банке, каков результат сложны

In [4]:
del_tokenizer = DelimiterTokenizer()


In [5]:
#get raw lexicon list
ru_lex = list(pd.read_csv("https://raw.githubusercontent.com/aigents/aigents-java/master/lexicon_russian.txt",sep='\t',header=None).to_records(index=False))
print(len(ru_lex))

#debug raw lexicon
print(max(ru_lex,key=lambda item:item[1]))
ru_lex_dict = weightedlist2dict(ru_lex,lower=False) # no case-insensitive merge
print(len(ru_lex_dict))
print(ru_lex_dict['не'])
print(ru_lex_dict['Не'])
print(ru_lex_dict['НЕ'])

# merge and get top weight
ru_lex_dict = weightedlist2dict(ru_lex,lower=True) # with case-insensitive merge
print(len(ru_lex_dict))
print(ru_lex_dict['не'])
top_weight = max([ru_lex_dict[key] for key in ru_lex_dict],key=lambda item:item)
print(top_weight)

# add delimiters to the list
ru_lex_delimited = ru_lex + [(i, top_weight) for i in list(delimiters)]
print(len(delimiters))
print(len(ru_lex_delimited)) 


100000
('не', 3164900)
100000
3164900
218183
2075
91396
3385158
3385158
34
100034


In [6]:
# no delimiters
filter_thresholds = [0,0.00001,0.0001,0.001,0.01]
for t in filter_thresholds:
    lex = listofpairs_compress_with_loss(ru_lex,t) if t > 0 else ru_lex
    ru_lex0_tokenizer = LexiconIndexedTokenizer(lexicon=lex,sortmode=0,cased=True)
    ru_lex1_tokenizer = LexiconIndexedTokenizer(lexicon=lex,sortmode=1,cased=True)
    ru_lex2_tokenizer = LexiconIndexedTokenizer(lexicon=lex,sortmode=2,cased=True)
    print(t,ru_lex0_tokenizer.count_params())
    print(evaluate_tokenizer_f1(test_texts,del_tokenizer,ru_lex0_tokenizer,debug=False))#sort by len
    print(evaluate_tokenizer_f1(test_texts,del_tokenizer,ru_lex1_tokenizer,debug=False))#sort by freq
    print(evaluate_tokenizer_f1(test_texts,del_tokenizer,ru_lex2_tokenizer,debug=False))#sort by len and freq
    print()


0 91396
0.79
0.36
0.75

1e-05 91396
0.79
0.36
0.75

0.0001 46290
0.73
0.36
0.71

0.001 5391
0.49
0.31
0.48

0.01 399
0.29
0.22
0.29



In [7]:
# with delimiters
filter_thresholds = [0,0.00001,0.0001,0.001,0.01]
for t in filter_thresholds:
    lex = listofpairs_compress_with_loss(ru_lex_delimited,t) if t > 0 else ru_lex_delimited
    ru_lex0_tokenizer = LexiconIndexedTokenizer(lexicon=lex,sortmode=0,cased=True)
    ru_lex1_tokenizer = LexiconIndexedTokenizer(lexicon=lex,sortmode=1,cased=True)
    ru_lex2_tokenizer = LexiconIndexedTokenizer(lexicon=lex,sortmode=2,cased=True)
    print(t,ru_lex0_tokenizer.count_params())
    print(evaluate_tokenizer_f1(test_texts,del_tokenizer,ru_lex0_tokenizer,debug=False))#sort by len
    print(evaluate_tokenizer_f1(test_texts,del_tokenizer,ru_lex1_tokenizer,debug=False))#sort by freq
    print(evaluate_tokenizer_f1(test_texts,del_tokenizer,ru_lex2_tokenizer,debug=False))#sort by len and freq
    print()


0 91430
0.94
0.56
0.91

1e-05 91430
0.94
0.56
0.91

0.0001 43863
0.89
0.57
0.87

0.001 4994
0.72
0.56
0.71

0.01 408
0.65
0.6
0.65



In [8]:
ru_lex0_tokenizer = LexiconIndexedTokenizer(lexicon=ru_lex_delimited,sortmode=0,cased=True)
for text in test_texts:
    expected = del_tokenizer.tokenize(text)
    actual = ru_lex0_tokenizer.tokenize(text)
    f1 = calc_f1(expected,actual)
    if f1 < 1:
        print(expected)
        print(actual)
        print(round(f1,2))


['На', ' ', 'самом', ' ', 'деле', ',', ' ', 'это', ' ', 'явление', ' ', 'действительно', ' ', 'очень', ' ', 'распространено', ',', ' ', 'например', ',', ' ', 'для', ' ', 'страхования', ' ', 'от', ' ', 'несчастных', ' ', 'случаев', ',', ' ', 'чем', ' ', 'больше', ' ', 'вы', ' ', 'покупаете', ',', ' ', 'тем', ' ', 'больше', ' ', 'страхуете', '.']
['На', ' ', 'самом', ' ', 'деле', ',', ' ', 'это', ' ', 'явление', ' ', 'действительно', ' ', 'очень', ' ', 'распространено', ',', ' ', 'например', ',', ' ', 'для', ' ', 'страхования', ' ', 'от', ' ', 'несчастных', ' ', 'случаев', ',', ' ', 'чем', ' ', 'больше', ' ', 'вы', ' ', 'покупаете', ',', ' ', 'тем', ' ', 'больше', ' ', 'страху', 'е', 'те', '.']
0.96
['Послушайте', ',', ' ', 'я', ' ', 'не', ' ', 'знаю', ',', ' ', 'слышали', ' ', 'ли', ' ', 'вы', ' ', 'когда-нибудь', ' ', 'об', ' ', 'этом', ',', ' ', 'это', ' ', 'страховка', ' ', 'в', ' ', 'Сбере', '.']
['Послушайте', ',', ' ', 'я', ' ', 'не', ' ', 'знаю', ',', ' ', 'слышали', ' ', 'ли', '

['Например', ',', ' ', 'если', ' ', 'вы', ' ', 'столкнулись', ' ', 'с', ' ', 'дорожно-транспортным', ' ', 'происшествием', ',', ' ', 'кого-то', ' ', 'сбили', ',', ' ', 'и', ' ', 'вам', ',', ' ', 'следовательно', ',', ' ', 'нужно', ' ', 'заплатить', ' ', 'определенную', ' ', 'сумму', '.']
['Например', ',', ' ', 'если', ' ', 'вы', ' ', 'столкнулись', ' ', 'с', ' ', 'дор', 'ож', 'но', '-', 'транспортным', ' ', 'происшествием', ',', ' ', 'кого-то', ' ', 'сбили', ',', ' ', 'и', ' ', 'вам', ',', ' ', 'следовательно', ',', ' ', 'нужно', ' ', 'заплатить', ' ', 'определенную', ' ', 'сумму', '.']
0.92
['Например', ',', ' ', 'очень', ' ', 'простой', ' ', 'пример', ',', ' ', 'такой', ' ', 'как', ' ', 'землетрясение', ' ', 'или', ' ', 'пожар', ',', ' ', 'вы', ' ', 'верите', ',', ' ', 'что', ' ', 'страховая', ' ', 'компания', ' ', 'возместит', ' ', 'вам', ' ', 'компенсацию', '?']
['Например', ',', ' ', 'очень', ' ', 'простой', ' ', 'пример', ',', ' ', 'такой', ' ', 'как', ' ', 'землетрясение', ' ', 

## Explore validity of the discovered lexicon


In [6]:
ru_lex_delimited_dict = weightedlist2dict(ru_lex_delimited,lower=True)

In [7]:
# use SOTA
base = FreedomTokenizer(name='data/models/rusage_test_chars_7a',max_n=7,mode='chars',debug=False)
test_tokenizer = FreedomBasedTokenizer(base,'ddf-','ddf+')
test_tokenizer.set_options(nlist = [1], threshold=0.4) # expected F1=1.00


In [8]:
expected = {}
actual = {}
f1 = evaluate_tokenizer_f1(test_texts,del_tokenizer,test_tokenizer,expected_collector=expected,actual_collector=actual)
print(f1)


1.0


In [13]:
#collected tokens

print('total relevant false precision')

expected_count = sum([expected[key] for key in expected])
relevant_count = sum([expected[key] for key in expected if key.lower() in ru_lex_delimited_dict])
irrelevant_count = sum([expected[key] for key in expected if not key.lower() in ru_lex_delimited_dict])
print(expected_count,relevant_count,irrelevant_count,relevant_count/expected_count,(relevant_count+160)/expected_count)

actual_count = sum([actual[key] for key in actual])
relevant_count = sum([actual[key] for key in actual if key.lower() in ru_lex_delimited_dict])
irrelevant_count = sum([actual[key] for key in actual if not key.lower() in ru_lex_delimited_dict])
print(actual_count,relevant_count,irrelevant_count,relevant_count/actual_count,(relevant_count+161)/actual_count)


total relevant false precision
2297 2137 160 0.9303439268611232 1.0
2298 2137 161 0.9299390774586597 1.0


In [10]:
#delimiter-based tokenizer
misses = sorted([(key,expected[key]) for key in expected if not key.lower() in ru_lex_delimited_dict],key = lambda x: x[1],reverse=True)
misses


[('в', 27),
 ('и', 22),
 ('у', 11),
 ('с', 8),
 ('а', 5),
 ('я', 5),
 ('—', 4),
 ('к', 4),
 ('Я', 3),
 ('кредитная', 3),
 ('ценообразование', 2),
 ('С', 2),
 ('500', 2),
 ('договорным', 2),
 ('застрахованным', 2),
 ('В', 1),
 ('300', 1),
 ('страхуете', 1),
 ('Сбере', 1),
 ('инвестицией', 1),
 ('Альфе', 1),
 ('депонирован', 1),
 ('казначейские', 1),
 ('продайте', 1),
 ('погасите', 1),
 ('инвестиционное', 1),
 ('зарегистрироваться', 1),
 ('интернет-банкинге', 1),
 ('кредитовать', 1),
 ('гарантированную', 1),
 ('2,8', 1),
 ('страховке', 1),
 ('стабильную', 1),
 ('срочному', 1),
 ('депозиту', 1),
 ('менеджменту', 1),
 ('инвестировании', 1),
 ('равносильна', 1),
 ('квалификационный', 1),
 ('брокера-дилера', 1),
 ('Составьте', 1),
 ('выберите', 1),
 ('взносом', 1),
 ('20', 1),
 ('И', 1),
 ('Застраховаться', 1),
 ('стандартизированным', 1),
 ('80%', 1),
 ('дебетовая', 1),
 ('оформляете', 1),
 ('150%', 1),
 ('базовое', 1),
 ('распродажа', 1),
 ('дорожно-транспортным', 1),
 ('возместит', 1),
 (

In [12]:
#freedom-based tokenizer
misses = sorted([(key,actual[key]) for key in actual if not key.lower() in ru_lex_delimited_dict],key = lambda x: x[1],reverse=True)
misses


[('в', 27),
 ('и', 22),
 ('у', 11),
 ('с', 8),
 ('а', 5),
 ('я', 5),
 ('—', 4),
 ('к', 4),
 ('Я', 3),
 ('кредитная', 3),
 ('ценообразование', 2),
 ('С', 2),
 ('500', 2),
 ('договорным', 2),
 ('застрахованным', 2),
 ('В', 1),
 ('300', 1),
 ('страхуете', 1),
 ('Сбере', 1),
 ('инвестицией', 1),
 ('Альфе', 1),
 ('депонирован', 1),
 ('казначейские', 1),
 ('продайте', 1),
 ('погасите', 1),
 ('инвестиционное', 1),
 ('зарегистрироваться', 1),
 ('интернет-банкинге', 1),
 ('кредитовать', 1),
 ('гарантированную', 1),
 ('2', 1),
 (',8', 1),
 ('страховке', 1),
 ('стабильную', 1),
 ('срочному', 1),
 ('депозиту', 1),
 ('менеджменту', 1),
 ('инвестировании', 1),
 ('равносильна', 1),
 ('квалификационный', 1),
 ('брокера-дилера', 1),
 ('Составьте', 1),
 ('выберите', 1),
 ('взносом', 1),
 ('20', 1),
 ('И', 1),
 ('Застраховаться', 1),
 ('стандартизированным', 1),
 ('80%', 1),
 ('дебетовая', 1),
 ('оформляете', 1),
 ('150%', 1),
 ('базовое', 1),
 ('распродажа', 1),
 ('дорожно-транспортным', 1),
 ('возмести